In [1]:
import sqlite3

# Database file
db_path = "violator_data.db"

# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS violators (
    face_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    phone TEXT NOT NULL,
    vehicle TEXT NOT NULL
);
"""
cursor.execute(create_table_query)

In [2]:
sample_data = [
    ("mypic", "Aarju Kumar", "9876543210", "MH12AB1234"),
    ("mypic2", "aarju2", "9123456789", "KA03CD5678"),
    ("Tamanna_stored_faces", "Tamanna Dhaker", "9123456789", "DL03CD5678"),
]

insert_query = "INSERT OR IGNORE INTO violators (face_id, name, phone, vehicle) VALUES (?, ?, ?, ?)"
cursor.executemany(insert_query, sample_data)

# Commit changes and close connection
conn.commit()
conn.close()

print("Database and table created successfully with sample data!")

Database and table created successfully with sample data!


In [ ]:
import sys
import os
import cv2
import face_recognition
import sqlite3
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QPushButton, QVBoxLayout,
    QHBoxLayout, QFileDialog, QWidget, QMessageBox, QTableWidget,
    QTableWidgetItem
)
from PyQt5.QtCore import Qt


class HelmetViolationApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Helmet Violation Detection")
        self.setGeometry(100, 100, 1200, 800)

        # Central widget and layout
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout()

        # Status label
        self.status_label = QLabel("Ready")
        layout.addWidget(self.status_label)

        # Process button
        self.process_button = QPushButton("Process Violations", self)
        self.process_button.clicked.connect(self.process_violations)
        layout.addWidget(self.process_button)

        # Table for displaying violators
        self.table = QTableWidget(self)
        self.table.setColumnCount(3)
        self.table.setHorizontalHeaderLabels(["Name", "Phone", "Vehicle"])
        layout.addWidget(self.table)

        central_widget.setLayout(layout)

        # Initialize paths
        self.violations_folder = "helmet_violations"
        self.bbox_folder = os.path.join(self.violations_folder, "bbox_data")
        self.stored_faces_dir = "stored_faces"
        self.db_path = "violator_data.db"

    def process_violations(self):
        if not os.path.exists(self.violations_folder) or not os.path.exists(self.bbox_folder):
            QMessageBox.critical(self, "Error", "Violation folders are missing!")
            return

        self.status_label.setText("Processing violations...")
        self.table.setRowCount(0)

        known_faces, known_names = self.load_stored_faces()

        for file_name in os.listdir(self.bbox_folder):
            if not file_name.endswith(".txt"):
                continue

            # Extract the corresponding image file
            base_name = os.path.splitext(file_name)[0]
            image_path = os.path.join(self.violations_folder, f"{base_name}.jpg")
            bbox_path = os.path.join(self.bbox_folder, file_name)

            if not os.path.exists(image_path):
                continue

            # Crop faces from the image
            cropped_faces = self.crop_faces(image_path, bbox_path)

            for face in cropped_faces:
                matched_name = self.match_face(face, known_faces, known_names)
                if matched_name:
                    self.add_violator_to_table(matched_name)

        self.status_label.setText("Processing complete.")

    def load_stored_faces(self):
        known_faces = []
        known_names = []

        # List all files in the stored_faces directory
        for face_file in os.listdir(self.stored_faces_dir):
            # Skip non-image files
            if not face_file.lower().endswith((".jpg", ".jpeg", ".png")):
                continue
            
            face_path = os.path.join(self.stored_faces_dir, face_file)
            face_image = face_recognition.load_image_file(face_path)
            
            # Ensure at least one face encoding is detected
            face_encodings = face_recognition.face_encodings(face_image)
            if face_encodings:
                known_faces.append(face_encodings[0])
                # Use the file name (without extension) as the person's name
                known_names.append(os.path.splitext(face_file)[0])
        
        return known_faces, known_names


    def crop_faces(self, image_path, bbox_path):
        image = cv2.imread(image_path)
        cropped_faces = []

        with open(bbox_path, "r") as bbox_file:
            for line in bbox_file:
                # Unpack the values; ignore the confidence score
                class_id, x_center, y_center, width, height, _ = map(float, line.strip().split())
                if int(class_id) == 1:  # Without helmet class
                    h, w, _ = image.shape
                    x1 = int((x_center - width / 2) * w)
                    y1 = int((y_center - height / 2) * h)
                    x2 = int((x_center + width / 2) * w)
                    y2 = int((y_center + height / 2) * h)
                    cropped_faces.append(image[y1:y2, x1:x2])

        return cropped_faces


    def match_face(self, face_image, known_faces, known_names):
        # Convert BGR to RGB
        rgb_face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
        
        # Get face encodings
        face_encoding = face_recognition.face_encodings(rgb_face_image)
        if not face_encoding:
            return None

        # Compare face encoding with known faces
        matches = face_recognition.compare_faces(known_faces, face_encoding[0])
        if True in matches:
            match_index = matches.index(True)
            return known_names[match_index]
        return None


    def add_violator_to_table(self, face_id):
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT name, phone, vehicle FROM violators WHERE face_id = ?", (face_id,))
            record = cursor.fetchone()
            conn.close()

            if record:
                row_position = self.table.rowCount()
                self.table.insertRow(row_position)
                for column, data in enumerate(record):
                    self.table.setItem(row_position, column, QTableWidgetItem(str(data)))
        except sqlite3.Error as e:
            QMessageBox.critical(self, "Database Error", str(e))


if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = HelmetViolationApp()
    main_window.show()
    sys.exit(app.exec_())


2024-11-22 12:19:29.376 Python[29387:1029676] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


SystemExit: 0

/Users/aarjukumar/Aarju_Kumar/Development/BikeHelmetDetection_FaceDetection/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 